# Joining Data in SQL

> the power of joining tables while exploring interesting features of countries and their cities throughout the world

- author: Victor Omondi
- toc: true
- comments: true
- categories: [joins, sql]
- image: images/jds-shield-png

# Overview

We'll explore the power of joining tables while exploring interesting features of countries and their cities throughout the world. We will use inner and outer joins, as well as self joins, semi joins, anti joins and cross joins—fundamental tools in any PostgreSQL wizard's toolbox.

# Setup 

In [1]:
import pandas as pd
%load_ext sql
%sql sqlite://

'Connected: @None'

# Introduction to joins

We'll be exploring the concept of joining tables, and will explore the different ways to enrich queries using inner joins and self joins. We'll also see how to use the case statement to split up a field into different categories.

## Introduction to INNER JOIN

### prime_ministers table

In [2]:
prime_ministers = pd.read_csv("datasets/leaders/prime_ministers.csv")
prime_ministers.head()

,country,continent,prime_minister
0,Egypt,Africa,Sherif Ismail
1,Portugal,Europe,Antonio Costa
2,Vietnam,Asia,Nguyen Xuan Phuc
3,Haiti,North America,Jack Guy Lafontant
4,India,Asia,Narendra Modi


In [3]:
presidents = pd.read_csv("datasets/leaders/presidents.csv")
presidents.head()

,country,continent,president
0,Egypt,Africa,Abdel Fattah el-Sisi
1,Portugal,Europe,Marcelo Rebelo de Sousa
2,Haiti,North America,Jovenel Moise
3,Uruguay,South America,Jose Mujica
4,Liberia,Africa,Ellen Johnson Sirleaf


In [4]:
%sql DROP TABLE IF EXISTS prime_ministers;
%sql PERSIST prime_ministers;
%sql DROP TABLE IF EXISTS presidents;
%sql PERSIST presidents;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://


'Persisted presidents'

### INNER JOIN in SQL

In [5]:
%%sql
SELECT p1.country, p1.continent, prime_minister, president
    FROM prime_ministers AS p1
    INNER JOIN presidents AS p2
    ON p1.country = p2.country

 * sqlite://
Done.


country,continent,prime_minister,president
Egypt,Africa,Sherif Ismail,Abdel Fattah el-Sisi
Portugal,Europe,Antonio Costa,Marcelo Rebelo de Sousa
Vietnam,Asia,Nguyen Xuan Phuc,Tran Dai Quang
Haiti,North America,Jack Guy Lafontant,Jovenel Moise


We'll be working with the `countries` database containing information about the most populous world cities as well as country-level economic data, population data, and geographic data. This `countries` database also contains information on languages spoken in each country.

In [6]:
cities = pd.read_csv("datasets/countries/cities.csv")
cities.head()

,name,country_code,city_proper_pop,metroarea_pop,urbanarea_pop
0,Abidjan,CIV,4765000,NaN,4765000
1,Abu Dhabi,ARE,1145000,NaN,1145000
2,Abuja,NGA,1235880,6000000.0,1235880
3,Accra,GHA,2070463,4010054.0,2070463
4,Addis Ababa,ETH,3103673,4567857.0,3103673


In [7]:
countries = pd.read_csv("datasets/countries/countries.csv")
countries.head()

,code,country_name,continent,region,surface_area,indep_year,local_name,gov_form,capital,cap_long,cap_lat
0,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,Afganistan/Afqanestan,Islamic Emirate,Kabul,69.17610,34.5228
1,NLD,Netherlands,Europe,Western Europe,41526.0,1581.0,Nederland,Constitutional Monarchy,Amsterdam,4.89095,52.3738
2,ALB,Albania,Europe,Southern Europe,28748.0,1912.0,Shqiperia,Republic,Tirane,19.81720,41.3317
3,DZA,Algeria,Africa,Northern Africa,2381740.0,1962.0,Al-Jazair/Algerie,Republic,Algiers,3.05097,36.7397
4,ASM,American Samoa,Oceania,Polynesia,199.0,NaN,Amerika Samoa,US Territory,Pago Pago,-170.69100,-14.2846


In [8]:
%sql DROP TABLE IF EXISTS cities;
%sql DROP TABLE IF EXISTS countries;
%sql PERSIST cities;
%sql PERSIST countries;

 * sqlite://
Done.
 * sqlite://
Done.
 * sqlite://
 * sqlite://


'Persisted countries'

In [9]:
%%sql
-- Select all columns from cities
SELECT *
    FROM cities
    LIMIT 5;

 * sqlite://
Done.


index,name,country_code,city_proper_pop,metroarea_pop,urbanarea_pop
0,Abidjan,CIV,4765000,None,4765000
1,Abu Dhabi,ARE,1145000,None,1145000
2,Abuja,NGA,1235880,6000000.0,1235880
3,Accra,GHA,2070463,4010054.0,2070463
4,Addis Ababa,ETH,3103673,4567857.0,3103673


In [10]:
%%sql
SELECT * 
FROM cities
    -- 1. Inner join to countries
    INNER JOIN countries
    -- 2. Match on the country codes
    ON cities.country_code = countries.code
    LIMIT 5;

 * sqlite://
Done.


index,name,country_code,city_proper_pop,metroarea_pop,urbanarea_pop,index_1,code,country_name,continent,region,surface_area,indep_year,local_name,gov_form,capital,cap_long,cap_lat
0,Abidjan,CIV,4765000,None,4765000,133,CIV,Cote d'Ivoire,Africa,Western Africa,322463.0,1960.0,Cote dIvoire,Republic,Yamoussoukro,-4.0305,5.332000000000001
1,Abu Dhabi,ARE,1145000,None,1145000,8,ARE,United Arab Emirates,Asia,Middle East,83600.0,1971.0,Al-Imarat al-´Arabiya al-Muttahida,Emirate Federation,Abu Dhabi,54.3705,24.4764
2,Abuja,NGA,1235880,6000000.0,1235880,131,NGA,Nigeria,Africa,Western Africa,923768.0,1960.0,Nigeria,Federal Republic,Abuja,7.48906,9.05804
3,Accra,GHA,2070463,4010054.0,2070463,52,GHA,Ghana,Africa,Western Africa,238533.0,1957.0,Ghana,Republic,Accra,-0.20795,5.57045
4,Addis Ababa,ETH,3103673,4567857.0,3103673,45,ETH,Ethiopia,Africa,Eastern Africa,1104300.0,-1000.0,YeItyop´iya,Republic,Addis Ababa,38.7468,9.022739999999999


In [11]:
%%sql
-- 1. Select name fields (with alias) and region 
SELECT cities.name AS city, countries.country_name AS country, countries.region
    FROM cities
    INNER JOIN countries
    ON cities.country_code = countries.code
    LIMIT 5;

 * sqlite://
Done.


city,country,region
Abidjan,Cote d'Ivoire,Western Africa
Abu Dhabi,United Arab Emirates,Middle East
Abuja,Nigeria,Western Africa
Accra,Ghana,Western Africa
Addis Ababa,Ethiopia,Eastern Africa


Instead of writing the full table name, we can use table aliasing as a shortcut. For tables we also use AS to add the alias immediately after the table name with a space. To select a field in the query that appears in multiple tables, we'll need to identify which table/table alias we're referring to by using a `.` in the `SELECT` statement.

We'll now explore a way to get data from both the `countries` and `economies` tables to examine the inflation rate for both 2010 and 2015.

In [12]:
economies = pd.read_csv("datasets/countries/economies.csv")
economies.head()

,econ_id,code,year,income_group,gdp_percapita,gross_savings,inflation_rate,total_investment,unemployment_rate,exports,imports
0,1,AFG,2010,Low income,539.667,37.133,2.179,30.402,NaN,46.394,24.381
1,2,AFG,2015,Low income,615.091,21.466,-1.549,18.602,NaN,-49.110,-7.294
2,3,AGO,2010,Upper middle income,3599.270,23.534,14.480,14.433,NaN,-3.266,-21.076
3,4,AGO,2015,Upper middle income,3876.200,-0.425,10.287,9.552,NaN,6.721,-21.778
4,5,ALB,2010,Upper middle income,4098.130,20.011,3.605,31.305,14.0,10.645,-8.013


In [13]:
%sql DROP TABLE IF EXISTS economies
%sql PERSIST economies;

 * sqlite://
Done.
 * sqlite://


'Persisted economies'

In [14]:
%%sql
-- 3. Select fields with aliases
SELECT c.code AS country_code, country_name, year, inflation_rate
    FROM countries AS c
    -- 1. Join to economies (alias e)
    INNER JOIN economies AS e
    -- 2. Match on code
    ON c.code = e.code
    LIMIT 5;

 * sqlite://
Done.


country_code,country_name,year,inflation_rate
AFG,Afghanistan,2010,2.1790000000000003
AFG,Afghanistan,2015,-1.5490000000000002
NLD,Netherlands,2010,0.932
NLD,Netherlands,2015,0.22
ALB,Albania,2010,3.605


In [15]:
populations = pd.read_csv("datasets/countries/populations.csv")
populations.head()

,pop_id,country_code,year,fertility_rate,life_expectancy,size
0,20,ABW,2010,1.704,74.953537,101597.0
1,19,ABW,2015,1.647,75.573585,103889.0
2,2,AFG,2010,5.746,58.970829,27962207.0
3,1,AFG,2015,4.653,60.717171,32526562.0
4,12,AGO,2010,6.416,50.654171,21219954.0


In [16]:
%sql DROP TABLE IF EXISTS populations;
%sql PERSIST populations;

 * sqlite://
Done.
 * sqlite://


'Persisted populations'

Now, for each country, we want to get the country name, its region, and the fertility rate and unemployment rate for both 2010 and 2015.

In [17]:
%%sql
-- 4. Select fields
SELECT code, country_name, region, year, fertility_rate
    -- 1. From countries (alias as c)
    FROM countries as c
    -- 2. Join with populations (as p)
    INNER JOIN populations as p
    -- 3. Match on country code
    ON code = country_code
    LIMIT 5;

 * sqlite://
Done.


code,country_name,region,year,fertility_rate
AFG,Afghanistan,Southern and Central Asia,2010,5.746
AFG,Afghanistan,Southern and Central Asia,2015,4.6530000000000005
NLD,Netherlands,Western Europe,2010,1.79
NLD,Netherlands,Western Europe,2015,1.71
ALB,Albania,Southern Europe,2010,1.663


In [18]:
%%sql
-- 6. Select fields
SELECT c.code, country_name, region, e.year, fertility_rate, unemployment_rate
    -- 1. From countries (alias as c)
    FROM countries AS c
    -- 2. Join to populations (as p)
    INNER JOIN populations AS p
    -- 3. Match on country code
    ON c.code = p.country_code
    -- 4. Join to economies (as e)
    INNER JOIN economies as e
    -- 5. Match on country code
    ON c.code = e.code
    LIMIT 5;

 * sqlite://
Done.


code,country_name,region,year,fertility_rate,unemployment_rate
AFG,Afghanistan,Southern and Central Asia,2010,4.6530000000000005,None
AFG,Afghanistan,Southern and Central Asia,2015,4.6530000000000005,None
AFG,Afghanistan,Southern and Central Asia,2010,5.746,None
AFG,Afghanistan,Southern and Central Asia,2015,5.746,None
NLD,Netherlands,Western Europe,2010,1.71,4.995


In [19]:
%%sql
-- 6. Select fields
SELECT c.code, country_name, region, e.year, fertility_rate, unemployment_rate
    -- 1. From countries (alias as c)
    FROM countries AS c
    -- 2. Join to populations (as p)
    INNER JOIN populations AS p
    -- 3. Match on country code
    ON c.code = p.country_code
     -- 4. Join to economies (as e)
    INNER JOIN economies AS e
    -- 5. Match on country code and year
    ON c.code = e.code AND e.year=p.year
    LIMIT 5;

 * sqlite://
Done.


code,country_name,region,year,fertility_rate,unemployment_rate
AFG,Afghanistan,Southern and Central Asia,2010,5.746,None
AFG,Afghanistan,Southern and Central Asia,2015,4.6530000000000005,None
NLD,Netherlands,Western Europe,2010,1.79,4.995
NLD,Netherlands,Western Europe,2015,1.71,6.891
ALB,Albania,Southern Europe,2010,1.663,14.0


## INNER JOIN via USING


`INNER JOIN `requires a specification of the key field (or fields) in each table.

When joining tables with a common field name You can use `USING` as a shortcut

In [20]:
countries.rename(columns={"country_name":"name"}, inplace=True)
%sql DROP TABLE IF EXISTS countries;
%sql PERSIST countries; 

 * sqlite://
Done.
 * sqlite://


'Persisted countries'

In [21]:
languages = pd.read_csv("datasets/countries/languages.csv")
languages.head()

,lang_id,code,name,percent,official
0,1,AFG,Dari,50.0,True
1,2,AFG,Pashto,35.0,True
2,3,AFG,Turkic,11.0,False
3,4,AFG,Other,4.0,False
4,5,ALB,Albanian,98.8,True


In [22]:
%sql DROP TABLE IF EXISTS languages;
%sql PERSIST languages

 * sqlite://
Done.
 * sqlite://


'Persisted languages'

In [23]:
%%sql
-- 4. Select fields
SELECT c.name AS country, continent, l.name AS language, official
  -- 1. From countries (alias as c)
  FROM countries c
  -- 2. Join to languages (as l)
  INNER JOIN languages l
    -- 3. Match using code
    USING(code)
    LIMIT 5;

 * sqlite://
Done.


country,continent,language,official
Afghanistan,Asia,Dari,1
Afghanistan,Asia,Other,0
Afghanistan,Asia,Pashto,1
Afghanistan,Asia,Turkic,0
Netherlands,Europe,Dutch,1


## Self-ish joins, just in CASE


### Join prime_ministers to itself?

In [24]:
%%sql
SELECT p1.country country1, p2.country country2, p1.continent
    FROM prime_ministers p1
    INNER JOIN prime_ministers p2
    ON p1.continent=p2.continent AND p1.country<>p2.country
    LIMIT(5)

 * sqlite://
Done.


country1,country2,continent
Portugal,Norway,Europe
Portugal,Spain,Europe
Vietnam,Brunei,Asia
Vietnam,India,Asia
Vietnam,Oman,Asia


### Creating indep_year_group in states

In [25]:
states = pd.read_csv("datasets/leaders/states.csv")
states.head()

,name,continent,indep_year,fert_rate,women_parli_perc
0,Australia,Oceania,1901,1.88,32.74
1,Brunei,Asia,1984,1.96,6.06
2,Chile,South America,1810,1.80,15.82
3,Egypt,Africa,1922,2.70,14.90
4,Haiti,North America,1804,3.03,2.74


In [26]:
%sql DROP TABLE IF EXISTS states;
%sql PERSIST states;

 * sqlite://
Done.
 * sqlite://


'Persisted states'

In [27]:
%%sql
SELECT name, continent, indep_year,
    CASE WHEN indep_year < 1900 THEN "Before 1900"
        WHEN indep_year <= 1930 THEN "Between 1900 AND 1930"
        ELSE "After 1930"
    END AS indep_year_group
    FROM states
    ORDER BY indep_year_group
    LIMIT 13;

 * sqlite://
Done.


name,continent,indep_year,indep_year_group
Brunei,Asia,1984,After 1930
India,Asia,1947,After 1930
Oman,Asia,1951,After 1930
Vietnam,Asia,1945,After 1930
Chile,South America,1810,Before 1900
Haiti,North America,1804,Before 1900
Liberia,Africa,1847,Before 1900
Portugal,Europe,1143,Before 1900
Spain,Europe,1492,Before 1900
Uruguay,South America,1828,Before 1900


### Self-join

We'll use the `populations` table to perform a self-join to calculate the percentage increase in population from 2010 to 2015 for each country code!

Since we'll be joining the `populations` table to itself, We will alias `populations as p1` and also `populations as p2`. 

> Important: This is good practice whenever we are aliasing and tables have the same first letter.


> Note: We are required to alias the tables with self-joins.

In [28]:
%%sql
SELECT p1.country_code,
       p1.size AS size2010, 
       p2.size AS size2015,
       -- 1. calculate growth_perc
       ((p2.size - p1.size)/p1.size * 100.0) AS growth_perc
-- 2. From populations (alias as p1)
FROM populations AS p1
  -- 3. Join to itself (alias as p2)
  INNER JOIN populations AS p2
    -- 4. Match on country code
    ON p1.country_code = p2.country_code
        -- 5. and year (with calculation)
        AND p1.year = p2.year - 5
        LIMIT 5;

 * sqlite://
Done.


country_code,size2010,size2015,growth_perc
ABW,101597.0,103889.0,2.255972125161176
AFG,27962207.0,32526562.0,16.3233002316305
AGO,21219954.0,25021974.0,17.917192468937493
ALB,2913021.0,2889167.0,-0.8188749754979453
AND,84419.0,70473.0,-16.519977730131842


### Case when and then

Often it's useful to look at a numerical field not as raw data, but instead as being in different categories or groups.

We can use `CASE` with `WHEN`, `THEN`, `ELSE`, and `END` to define a new grouping field.

In [29]:
%%sql
CREATE TABLE countries_plus AS
SELECT name, continent, code, surface_area,
    -- 1. First case
    CASE WHEN surface_area > 2000000 THEN 'large'
        -- 2. Second case
        WHEN surface_area > 350000 AND surface_area<=2000000 THEN "medium"
        -- 3. Else clause + end
        ELSE "small" END
        -- 4. Alias name
        AS geosize_group
-- 5. From table
FROM countries;

 * sqlite://
Done.


[]

In [30]:
%%sql
SELECT * 
    FROM countries_plus
    LIMIT 5;

 * sqlite://
Done.


name,continent,code,surface_area,geosize_group
Afghanistan,Asia,AFG,652090.0,medium
Netherlands,Europe,NLD,41526.0,small
Albania,Europe,ALB,28748.0,small
Algeria,Africa,DZA,2381740.0,large
American Samoa,Oceania,ASM,199.0,small


### Inner challenge

We will now explore the relationship between the size of a country in terms of surface area and in terms of population using grouping fields created with CASE.

In [31]:
%%sql
CREATE TABLE pop_plus AS
SELECT country_code, size,
  CASE WHEN size > 50000000
            THEN 'large'
       WHEN size > 1000000
            THEN 'medium'
       ELSE 'small' END
       AS popsize_group       
FROM populations
WHERE year = 2015;

SELECT * FROM pop_plus
    LIMIT 5;

 * sqlite://
Done.
Done.


country_code,size,popsize_group
ABW,103889.0,small
AFG,32526562.0,medium
AGO,25021974.0,medium
ALB,2889167.0,medium
AND,70473.0,small


In [32]:
%%sql
-- 5. Select fields
SELECT name, continent, geosize_group, popsize_group
-- 1. From countries_plus (alias as c)
FROM countries_plus AS c
  -- 2. Join to pop_plus (alias as p)
  INNER JOIN pop_plus AS p
    -- 3. Match on country code
    ON c.code = p.country_code
-- 4. Order the table    
ORDER BY geosize_group
LIMIT 5;

 * sqlite://
Done.


name,continent,geosize_group,popsize_group
Algeria,Africa,large,medium
Argentina,South America,large,medium
Australia,Oceania,large,medium
Brazil,South America,large,large
Greenland,North America,large,small


# Outer joins and cross joins

We'll come to grips with different kinds of outer joins. We'll explore how to gain further insights into our data through left joins, right joins, and full joins. In addition to outer joins, we'll also work with cross joins.



## LEFT and RIGHT JOINs

### The syntax of a LEFT JOIN

In [33]:
%%sql
SELECT p1.country, prime_minister, president
    FROM prime_ministers AS p1
    LEFT JOIN presidents AS p2
    ON p1.country = p2.country;

 * sqlite://
Done.


country,prime_minister,president
Egypt,Sherif Ismail,Abdel Fattah el-Sisi
Portugal,Antonio Costa,Marcelo Rebelo de Sousa
Vietnam,Nguyen Xuan Phuc,Tran Dai Quang
Haiti,Jack Guy Lafontant,Jovenel Moise
India,Narendra Modi,None
Australia,Malcolm Turnbull,None
Norway,Erna Solberg,None
Brunei,Hassanal Bolkiah,None
Oman,Qaboos bin Said al Said,None
Spain,Mariano Rajoy,None


### Left Join

Now we'll explore the differences between performing an inner join and a left join using the `cities` and `countries` tables.

We'll begin by performing an inner join with the `cities` table on the left and the `countries` table on the right.

We will then change the query to a left join.

In [34]:
%%sql
-- Select the city name (with alias), the country code,
-- the country name (with alias), the region,
-- and the city proper population
SELECT c1.name AS city, code, c2.name AS country,
       region, city_proper_pop
-- From left table (with alias)
FROM cities AS c1
  -- Join to right table (with alias)
  INNER JOIN countries AS c2
    -- Match on country code
    ON c1.country_code = c2.code
-- Order by descending country code
ORDER BY code DESC
LIMIT 10;

 * sqlite://
Done.


city,code,country,region,city_proper_pop
Harare,ZWE,Zimbabwe,Eastern Africa,1606000
Lusaka,ZMB,Zambia,Eastern Africa,1742979
Cape Town,ZAF,South Africa,Southern Africa,3740026
Durban,ZAF,South Africa,Southern Africa,3442361
Ekurhuleni,ZAF,South Africa,Southern Africa,3178470
Johannesburg,ZAF,South Africa,Southern Africa,4434827
Sana'a,YEM,Yemen,Middle East,1937451
Hanoi,VNM,Vietnam,Southeast Asia,6844100
Ho Chi Minh City,VNM,Vietnam,Southeast Asia,7681700
Caracas,VEN,Venezuela,South America,1943901


In [35]:
%%sql
SELECT c1.name AS city, code, c2.name AS country,
       region, city_proper_pop
FROM cities AS c1
  -- 1. Join right table (with alias)
  LEFT JOIN countries AS c2
    -- 2. Match on country code
    ON c1.country_code = c2.code
-- 3. Order by descending country code
ORDER BY code DESC
LIMIT 10;

 * sqlite://
Done.


city,code,country,region,city_proper_pop
Harare,ZWE,Zimbabwe,Eastern Africa,1606000
Lusaka,ZMB,Zambia,Eastern Africa,1742979
Cape Town,ZAF,South Africa,Southern Africa,3740026
Durban,ZAF,South Africa,Southern Africa,3442361
Ekurhuleni,ZAF,South Africa,Southern Africa,3178470
Johannesburg,ZAF,South Africa,Southern Africa,4434827
Sana'a,YEM,Yemen,Middle East,1937451
Hanoi,VNM,Vietnam,Southeast Asia,6844100
Ho Chi Minh City,VNM,Vietnam,Southeast Asia,7681700
Caracas,VEN,Venezuela,South America,1943901


> Note: The `INNER JOIN` version resulted in 230 records. The `LEFT JOIN` version returned 236 rows.

### Left join (2)
Next, we'll try out another example comparing an inner join to its corresponding left join. 

We will begin with an inner join on the `countries` table on the left with the languages table on the right. Then we'll change the code to a left join in the next bullet.

Note the use of multi-line comments here using /* and */

In [36]:
%%sql
/*
5. Select country name AS country, the country's local name,
the language name AS language, and
the percent of the language spoken in the country
*/
SELECT c.name AS country, local_name, l.name AS language, percent
-- 1. From left table (alias as c)
FROM countries AS c
  -- 2. Join to right table (alias as l)
  INNER JOIN languages AS l
    -- 3. Match on fields
    USING(code)
-- 4. Order by descending country
ORDER BY country DESC
LIMIT 10;

 * sqlite://
Done.


country,local_name,language,percent
Zimbabwe,Zimbabwe,Chewa,None
Zimbabwe,Zimbabwe,Chibarwe,None
Zimbabwe,Zimbabwe,English,None
Zimbabwe,Zimbabwe,Kalanga,None
Zimbabwe,Zimbabwe,Koisan,None
Zimbabwe,Zimbabwe,Nambya,None
Zimbabwe,Zimbabwe,Ndau,None
Zimbabwe,Zimbabwe,Ndebele,None
Zimbabwe,Zimbabwe,Shangani,None
Zimbabwe,Zimbabwe,Shona,None


In [37]:
%%sql
/*
5. Select country name AS country, the country's local name,
the language name AS language, and
the percent of the language spoken in the country
*/
SELECT c.name AS country, local_name, l.name AS language, percent
-- 1. From left table (alias as c)
FROM countries AS c
  -- 2. Join to right table (alias as l)
  LEFT JOIN languages AS l
    -- 3. Match on fields
    USING(code)
-- 4. Order by descending country
ORDER BY country DESC
LIMIT 10;

 * sqlite://
Done.


country,local_name,language,percent
Zimbabwe,Zimbabwe,Chewa,None
Zimbabwe,Zimbabwe,Chibarwe,None
Zimbabwe,Zimbabwe,English,None
Zimbabwe,Zimbabwe,Kalanga,None
Zimbabwe,Zimbabwe,Koisan,None
Zimbabwe,Zimbabwe,Nambya,None
Zimbabwe,Zimbabwe,Ndau,None
Zimbabwe,Zimbabwe,Ndebele,None
Zimbabwe,Zimbabwe,Shangani,None
Zimbabwe,Zimbabwe,Shona,None


> Note: The `INNER JOIN` version resulted in 909 records. The `LEFT JOIN` version returned 916 rows.

We will use `AVG()` in combination with left join to determine the average gross domestic product (GDP) per capita by region in 2010.

In [38]:
%%sql
-- Select fields
SELECT region, AVG(gdp_percapita) AS avg_gdp
-- From countries (alias as c)
FROM countries c
  -- Left join with economies (alias as e)
  LEFT JOIN economies e
    -- Match on code fields
    USING(code)
-- Focus on 2010
WHERE year = 2010
-- Group by region
GROUP BY region
-- Order by descending avg_gdp
ORDER BY avg_gdp DESC
LIMIT 10;

 * sqlite://
Done.


region,avg_gdp
Western Europe,58130.962857142855
Nordic Countries,57073.998
North America,47911.509999999995
Australia and New Zealand,44792.384999999995
British Islands,43588.33
Eastern Asia,26205.851666666666
Southern Europe,22926.410909090908
Middle East,18204.64176470588
Baltic Countries,12631.029999999999
Caribbean,11413.339461538462


### Right join
Right joins aren't as common as left joins. One reason why is that you can always write a right join as a left join.

In [39]:
%%sql
SELECT cities.name AS city, urbanarea_pop, countries.name AS country,
       indep_year, languages.name AS language, percent
FROM cities
  LEFT JOIN countries
    ON cities.country_code = countries.code
  LEFT JOIN languages
    ON countries.code = languages.code
ORDER BY city, language
LIMIT 10;

 * sqlite://
Done.


city,urbanarea_pop,country,indep_year,language,percent
Abidjan,4765000,Cote d'Ivoire,1960.0,French,None
Abidjan,4765000,Cote d'Ivoire,1960.0,Other,None
Abu Dhabi,1145000,United Arab Emirates,1971.0,Arabic,None
Abu Dhabi,1145000,United Arab Emirates,1971.0,English,None
Abu Dhabi,1145000,United Arab Emirates,1971.0,Hindi,None
Abu Dhabi,1145000,United Arab Emirates,1971.0,Persian,None
Abu Dhabi,1145000,United Arab Emirates,1971.0,Urdu,None
Abuja,1235880,Nigeria,1960.0,English,None
Abuja,1235880,Nigeria,1960.0,Fulani,None
Abuja,1235880,Nigeria,1960.0,Hausa,None


## FULL JOINs


We'll examine how results differ when using a full join versus using a left join versus using an inner join with the `countries` and `currencies` tables.

We will focus on the `North American` region and also where the name of the country is missing.

In [40]:
currencies = pd.read_csv("datasets/countries/currencies.csv")
currencies.head()

,curr_id,code,basic_unit,curr_code,frac_unit,frac_perbasic
0,1,AFG,Afghan afghani,AFN,Pul,100.0
1,2,ALB,Albanian lek,ALL,Qindarke,100.0
2,3,DZA,Algerian dinar,DZD,Santeem,100.0
3,4,AND,Euro,EUR,Cent,100.0
4,5,AGO,Angolan kwanza,AOA,Centimo,100.0


In [41]:
%sql DROP TABLE IF EXISTS currencies;
%sql PERSIST currencies;

 * sqlite://
Done.
 * sqlite://


'Persisted currencies'

In [42]:
%%sql
SELECT name AS country, code, region, basic_unit
-- 3. From countries
FROM countries
  -- 4. Join to currencies
  LEFT JOIN currencies
    -- 5. Match on code
    USING (code)
-- 1. Where region is North America or null
WHERE region = 'North America' OR region IS NULL
UNION
SELECT name AS country, code, region, basic_unit
-- 3. From countries
FROM currencies
  -- 4. Join to currencies
  LEFT JOIN countries
    -- 5. Match on code
    USING (code)
-- 1. Where region is North America or null
WHERE region = 'North America' OR region IS NULL
ORDER BY region;

 * sqlite://
Done.


country,code,region,basic_unit
None,AIA,None,East Caribbean dollar
None,CCK,None,Australian dollar
None,COK,None,New Zealand dollar
None,FLK,None,Falkland Islands pound
None,IOT,None,United States dollar
None,MSR,None,East Caribbean dollar
None,NIU,None,New Zealand dollar
None,ROM,None,Romanian leu
None,SGS,None,British pound
None,SHN,None,Saint Helena pound


In [43]:
%%sql
SELECT name AS country, code, region, basic_unit
-- 1. From countries
FROM countries
  -- 2. Join to currencies
  LEFT JOIN currencies
    -- 3. Match on code
    USING (code)
-- 4. Where region is North America or null
WHERE region = 'North America' OR region IS null
-- 5. Order by region
ORDER BY region;

 * sqlite://
Done.


country,code,region,basic_unit
Bermuda,BMU,North America,Bermudian dollar
Greenland,GRL,North America,None
Canada,CAN,North America,Canadian dollar
United States,USA,North America,United States dollar


In [44]:
%%sql
SELECT name AS country, code, region, basic_unit
FROM countries
  -- 1. Join to currencies
  INNER JOIN currencies
    USING (code)
-- 2. Where region is North America or null
WHERE region = 'North America' OR region IS NULL
-- 3. Order by region
ORDER BY region;

 * sqlite://
Done.


country,code,region,basic_unit
Bermuda,BMU,North America,Bermudian dollar
Canada,CAN,North America,Canadian dollar
United States,USA,North America,United States dollar


The FULL JOIN query returned 17 rows, the OUTER JOIN returned 4 rows, and the INNER JOIN only returned 3 rows. 

We'll now investigate a similar exercise to the last one, but this time focused on using a table with more records on the left than the right. We'll work with the `languages` and `countries` tables.

We'll Begin with a full join with `languages` on the left and `countries` on the right. 

In [45]:
%%sql
SELECT countries.name, code, languages.name AS language
-- 3. From languages
FROM languages
  -- 4. Join to countries
  LEFT JOIN countries
    -- 5. Match on code
    USING (code)
-- 1. Where countries.name starts with V or is null
WHERE countries.name LIKE 'V%' OR countries.name IS NULL
-- 2. Order by ascending countries.name
UNION
SELECT countries.name, code, languages.name AS language
-- 3. From languages
FROM countries
  -- 4. Join to countries
  LEFT JOIN languages
    -- 5. Match on code
    USING (code)
-- 1. Where countries.name starts with V or is null
WHERE countries.name LIKE 'V%' OR countries.name IS NULL
-- 2. Order by ascending countries.name
ORDER BY countries.name DESC
LIMIT 10;

 * sqlite://
Done.


name,code,language
"Virgin Islands, U.S.",VIR,None
"Virgin Islands, British",VGB,None
Vietnam,VNM,English
Vietnam,VNM,Other
Vietnam,VNM,Vietnamese
Venezuela,VEN,Spanish
Venezuela,VEN,indigenous
Vanuatu,VUT,Bislama
Vanuatu,VUT,English
Vanuatu,VUT,French


In [46]:
%%sql
SELECT countries.name, code, languages.name AS language
FROM languages
  -- 1. Join to countries
  LEFT JOIN countries
    -- 2. Match using code
    USING (code)
-- 3. Where countries.name starts with V or is null
WHERE countries.name LIKE 'V%' OR countries.name IS NULL
ORDER BY countries.name
LIMIT 10;

 * sqlite://
Done.


name,code,language
None,AIA,English
None,CXR,English
None,CXR,Chinese
None,CXR,Malay
None,CCK,Malay
None,CCK,English
None,COK,English
None,COK,Rarotongan
None,COK,Other
None,MSR,English


In [47]:
%%sql
SELECT countries.name, code, languages.name AS language
FROM languages
  -- 1. Join to countries
  INNER JOIN countries
    USING (code)
-- 2. Where countries.name starts with V or is null
WHERE countries.name LIKE 'V%' OR countries.name IS NULL
ORDER BY countries.name;

 * sqlite://
Done.


name,code,language
Vanuatu,VUT,Bislama
Vanuatu,VUT,English
Vanuatu,VUT,French
Vanuatu,VUT,Other
Vanuatu,VUT,Tribal Languages
Venezuela,VEN,Spanish
Venezuela,VEN,indigenous
Vietnam,VNM,English
Vietnam,VNM,Other
Vietnam,VNM,Vietnamese


## CROSSing the rubicon

### Pairing prime ministers with presidents

In [48]:
%%sql
SELECT prime_minister, president
    FROM prime_ministers AS p1
    CROSS JOIN presidents AS p2
    WHERE p1.continent IN ('North America', 'Oceania');

 * sqlite://
Done.


prime_minister,president
Jack Guy Lafontant,Abdel Fattah el-Sisi
Jack Guy Lafontant,Marcelo Rebelo de Sousa
Jack Guy Lafontant,Jovenel Moise
Jack Guy Lafontant,Jose Mujica
Jack Guy Lafontant,Ellen Johnson Sirleaf
Jack Guy Lafontant,Michelle Bachelet
Jack Guy Lafontant,Tran Dai Quang
Malcolm Turnbull,Abdel Fattah el-Sisi
Malcolm Turnbull,Marcelo Rebelo de Sousa
Malcolm Turnbull,Jovenel Moise


### A table of two cities

We'll explore languages potentially and most frequently spoken in the cities of Hyderabad, India and Hyderabad, Pakistan.

We will begin with a cross join with `cities AS c` on the left and `languages AS l `on the right. Then we will modify the query using an inner join.

In [49]:
%%sql
-- 4. Select fields
SELECT c.name AS city, l.name AS language
-- 1. From cities (alias as c)
FROM cities AS c        
  -- 2. Join to languages (alias as l)
  CROSS JOIN languages AS l
-- 3. Where c.name like Hyderabad
WHERE c.name LIKE 'Hyder%'
LIMIT 10;

 * sqlite://
Done.


city,language
Hyderabad (India),Dari
Hyderabad (India),Pashto
Hyderabad (India),Turkic
Hyderabad (India),Other
Hyderabad (India),Albanian
Hyderabad (India),Greek
Hyderabad (India),Other
Hyderabad (India),unspecified
Hyderabad (India),Arabic
Hyderabad (India),French


In [50]:
%%sql
-- 5. Select fields
SELECT c.name AS city, l.name AS language
-- 1. From cities (alias as c)
FROM cities AS c      
  -- 2. Join to languages (alias as l)
  INNER JOIN languages AS l
    -- 3. Match on country code
    ON l.code = c.country_code
-- 4. Where c.name like Hyderabad
WHERE c.name LIKE 'Hyder%';

 * sqlite://
Done.


city,language
Hyderabad (India),Assamese
Hyderabad (India),Bengali
Hyderabad (India),Gujarati
Hyderabad (India),Hindi
Hyderabad (India),Kannada
Hyderabad (India),Maithili
Hyderabad (India),Malayalam
Hyderabad (India),Marathi
Hyderabad (India),Oriya
Hyderabad (India),Other


In terms of life expectancy for 2010, determine the names of the lowest five countries and their regions.

In [51]:
%%sql
-- Select fields
SELECT c.name AS country, region, life_expectancy AS life_exp
-- From countries (alias as c)
FROM countries c
  -- Join to populations (alias as p)
  LEFT JOIN populations p
    -- Match on country code
    ON c.code = p.country_code
-- Focus on 2010
WHERE year=2010
-- Order by life_exp
ORDER BY life_exp
-- Limit to 5 records
LIMIT 5;

 * sqlite://
Done.


country,region,life_exp
American Samoa,Polynesia,None
Andorra,Southern Europe,None
"Virgin Islands, British",Caribbean,None
Cayman Islands,Caribbean,None
Dominica,Caribbean,None


# Set theory clauses

We'll explore more about set theory using Venn diagrams and get an introduction to union, union all, intersect, and except clauses. We'll finish by investigating semi joins and anti joins, which provide a nice introduction to subqueries.

## State of the UNION


### Set Theory Venn Diagrams

![image.png](datasets/images/venn.png)

![image.png](datasets/images/union.png "Union")

![image.png](datasets/images/union-all.png "Union All")

### monarchs table

In [52]:
monarchs = pd.read_csv("datasets/leaders/monarchs.csv")
monarchs.head()

,country,continent,monarch
0,Brunei,Asia,Hassanal Bolkiah
1,Oman,Asia,Qaboos bin Said al Said
2,Norway,Europe,Harald V
3,Spain,Europe,Felipe VI


In [53]:
%sql DROP TABLE IF EXISTS monarchs;
%sql PERSIST monarchs;

 * sqlite://
Done.
 * sqlite://


'Persisted monarchs'

In [54]:
%%sql
SELECT *
    FROM monarchs
    LIMIT 5;

 * sqlite://
Done.


index,country,continent,monarch
0,Brunei,Asia,Hassanal Bolkiah
1,Oman,Asia,Qaboos bin Said al Said
2,Norway,Europe,Harald V
3,Spain,Europe,Felipe VI


### All prime ministers and monarchs

In [55]:
%%sql
SELECT prime_minister AS leader, country
    FROM prime_ministers
UNION
SELECT monarch, country
    FROM monarchs
    ORDER BY country;


 * sqlite://
Done.


leader,country
Malcolm Turnbull,Australia
Hassanal Bolkiah,Brunei
Sherif Ismail,Egypt
Jack Guy Lafontant,Haiti
Narendra Modi,India
Erna Solberg,Norway
Harald V,Norway
Qaboos bin Said al Said,Oman
Antonio Costa,Portugal
Felipe VI,Spain


### UNION ALL with leaders

In [56]:
%%sql
SELECT prime_minister AS leader, country
    FROM prime_ministers
UNION ALL
SELECT monarch, country
    FROM monarchs
    ORDER BY country
    LIMIT 10;

 * sqlite://
Done.


leader,country
Malcolm Turnbull,Australia
Hassanal Bolkiah,Brunei
Hassanal Bolkiah,Brunei
Sherif Ismail,Egypt
Jack Guy Lafontant,Haiti
Narendra Modi,India
Erna Solberg,Norway
Harald V,Norway
Qaboos bin Said al Said,Oman
Qaboos bin Said al Said,Oman


In [57]:
economies2010 = pd.read_csv("datasets/countries/economies2010.csv")
economies2010.head()

,code,year,income_group,gross_savings
0,AFG,2010,Low income,37.133
1,AGO,2010,Upper middle income,23.534
2,ALB,2010,Upper middle income,20.011
3,ARE,2010,High income,27.073
4,ARG,2010,Upper middle income,17.361


In [58]:
%sql DROP TABLE IF EXISTS economies2010;
%sql PERSIST economies2010;

 * sqlite://
Done.
 * sqlite://


'Persisted economies2010'

In [59]:
economies2015 = pd.read_csv("datasets/countries/economies2015.csv")
%sql DROP TABLE IF EXISTS economies2015;
%sql PERSIST economies2015;
economies2015.head()

 * sqlite://
Done.
 * sqlite://


,code,year,income_group,gross_savings
0,AFG,2015,Low income,21.466
1,AGO,2015,Upper middle income,-0.425
2,ALB,2015,Upper middle income,13.840
3,ARE,2015,High income,34.106
4,ARG,2015,Upper middle income,14.111


In [60]:
%%sql
-- Select fields from 2010 table
SELECT *
  -- From 2010 table
  FROM economies2010
	-- Set theory clause
	UNION
-- Select fields from 2015 table
SELECT *
  -- From 2015 table
  FROM economies2015
-- Order by code and year
ORDER BY code, year
LIMIT 10;

 * sqlite://
Done.


index,code,year,income_group,gross_savings
0,AFG,2010,Low income,37.133
0,AFG,2015,Low income,21.465999999999998
1,AGO,2010,Upper middle income,23.534000000000002
1,AGO,2015,Upper middle income,-0.425
2,ALB,2010,Upper middle income,20.011
2,ALB,2015,Upper middle income,13.84
3,ARE,2010,High income,27.073
3,ARE,2015,High income,34.106
4,ARG,2010,Upper middle income,17.361
4,ARG,2015,Upper middle income,14.110999999999999


`UNION` can also be used to determine all occurrences of a field across multiple tables

In [61]:
%%sql
-- Select field
SELECT country_code
  -- From cities
  FROM cities
	-- Set theory clause
	UNION
-- Select field
SELECT code
  -- From currencies
  FROM currencies
-- Order by country_code
ORDER BY country_code
LIMIT 10;

 * sqlite://
Done.


country_code
ABW
AFG
AGO
AIA
ALB
AND
ARE
ARG
ARM
ATG


### Union all
As you saw, duplicates were removed by using `UNION`.

To include duplicates, you can use `UNION ALL`

In [62]:
%%sql
-- Select fields
SELECT code, year
  -- From economies
  FROM economies
	-- Set theory clause
	UNION ALL
-- Select fields
SELECT country_code, year
  -- From populations
  FROM populations
-- Order by code, year
ORDER BY code, year
LIMIT 10;

 * sqlite://
Done.


code,year
ABW,2010
ABW,2015
AFG,2010
AFG,2010
AFG,2015
AFG,2015
AGO,2010
AGO,2010
AGO,2015
AGO,2015


## INTERSECTional data science


![image.png](datasets/images/intersect.png "Intersect")

### Prime minister and president countries

In [63]:
%%sql
SELECT country
    FROM prime_ministers
INTERSECT
SELECT country
    FROM presidents;

 * sqlite://
Done.


country
Egypt
Haiti
Portugal
Vietnam


### INTERSECT on two fields

In [64]:
%%sql
SELECT country, prime_minister AS leader
    FROM prime_ministers
INTERSECT
SELECT country, president
    FROM presidents

 * sqlite://
Done.


country,leader


records in common for country code and year for the `economies` and `populations` tables.

In [65]:
%%sql
-- Select fields
SELECT code, year
  -- From economies
  FROM economies
	-- Set theory clause
	INTERSECT 
-- Select fields
SELECT country_code, year
  -- From populations
  FROM populations
-- Order by code and year
ORDER BY code, year
LIMIT 10;

 * sqlite://
Done.


code,year
AFG,2010
AFG,2015
AGO,2010
AGO,2015
ALB,2010
ALB,2015
ARE,2010
ARE,2015
ARG,2010
ARG,2015


which countries also have a city with the same name as their country name?

In [66]:
%%sql
-- Select fields
SELECT name
  -- From countries
  FROM countries
	-- Set theory clause
	INTERSECT
-- Select fields
SELECT name
  -- From cities
  FROM cities;

 * sqlite://
Done.


name
Hong Kong
Singapore


## EXCEPTional


### Monarchs that aren't prime ministers

In [67]:
%%sql
SELECT monarch, country 
    FROM monarchs
EXCEPT
SELECT prime_minister, country
    FROM prime_ministers

 * sqlite://
Done.


monarch,country
Felipe VI,Spain
Harald V,Norway


![image.png](datasets/images/except.png "Except")

### Except

The names of cities in `cities` which are not noted as capital cities in `countries` as a single field result.

Note that there are some countries in the world that are not included in the countries table, which will result in some cities not being labeled as capital cities when in fact they are.

In [68]:
%%sql
-- Select field
SELECT name
  -- From cities
  FROM cities
	-- Set theory clause
	EXCEPT
-- Select field
SELECT capital
  -- From countries
  FROM countries
-- Order by result
ORDER BY name
LIMIT 10;

 * sqlite://
Done.


name
Abidjan
Ahmedabad
Alexandria
Almaty
Auckland
Bandung
Barcelona
Barranquilla
Basra
Belo Horizonte


the names of capital cities that are not listed in the `cities` table.

In [69]:
%%sql
-- Select field
SELECT capital
  -- From countries
  FROM countries
	-- Set theory clause
	EXCEPT
-- Select field
SELECT name
  -- From cities
  FROM cities
-- Order by ascending capital
ORDER BY capital
LIMIT 10;

 * sqlite://
Done.


capital
None
Agana
Amman
Amsterdam
Andorra la Vella
Antananarivo
Apia
Ashgabat
Asmara
Astana


## Semi-joins and Anti-joins


### Building up to a semi-join

In [70]:
%%sql
SELECT name
    FROM states
    WHERE indep_year < 1800;

 * sqlite://
Done.


name
Portugal
Spain


### Another step towards the semi-join

In [71]:
%%sql
SELECT president, country, continent
    FROM presidents;

 * sqlite://
Done.


president,country,continent
Abdel Fattah el-Sisi,Egypt,Africa
Marcelo Rebelo de Sousa,Portugal,Europe
Jovenel Moise,Haiti,North America
Jose Mujica,Uruguay,South America
Ellen Johnson Sirleaf,Liberia,Africa
Michelle Bachelet,Chile,South America
Tran Dai Quang,Vietnam,Asia


### Finish the semi-join (an intro to subqueries)

In [72]:
%%sql
SELECT president, country, continent
    FROM presidents
    WHERE country IN   (
        SELECT name
            FROM states
            WHERE indep_year < 1800
    );

 * sqlite://
Done.


president,country,continent
Marcelo Rebelo de Sousa,Portugal,Europe


### An anti-join

In [73]:
%%sql
SELECT president, country, continent
    FROM presidents
    WHERE continent LIKE '%America' AND country NOT IN        
        (SELECT name
             FROM states
             WHERE indep_year < 1800);


 * sqlite://
Done.


president,country,continent
Jovenel Moise,Haiti,North America
Jose Mujica,Uruguay,South America
Michelle Bachelet,Chile,South America


![image.png](datasets/images/semi-anti-join.png "Semi Join and Anti Join")

languages spoken in the Middle East.

In [74]:
%%sql
-- Select distinct fields
SELECT DISTINCT(name)
  -- From languages
  FROM languages
-- Where in statement
WHERE code IN
  -- Subquery
  (SELECT code
   FROM countries
   WHERE region='Middle East')
-- Order by name
ORDER BY name;

 * sqlite://
Done.


name
Arabic
Aramaic
Armenian
Azerbaijani
Azeri
Baluchi
Bulgarian
Circassian
English
Farsi


### Diagnosing problems using anti-join
Another powerful join in SQL is the anti-join. It is particularly useful in identifying which records are causing an incorrect number of records to appear in join queries.

Our goal is to identify the currencies used in Oceanian countries!

In [75]:
%%sql
-- Select statement
SELECT COUNT(*) '# countries'
  -- From countries
  FROM countries
-- Where continent is Oceania
WHERE continent = 'Oceania';

 * sqlite://
Done.


# countries
19


In [76]:
%%sql
-- 5. Select fields (with aliases)
SELECT c1.code, c1.name, basic_unit currency
  -- 1. From countries (alias as c1)
  FROM countries c1
  	-- 2. Join with currencies (alias as c2)
  	INNER JOIN currencies
    -- 3. Match on code
    USING(code)
-- 4. Where continent is Oceania
WHERE continent='Oceania';

 * sqlite://
Done.


code,name,currency
AUS,Australia,Australian dollar
KIR,Kiribati,Australian dollar
MHL,Marshall Islands,United States dollar
NRU,Nauru,Australian dollar
PLW,Palau,United States dollar
PNG,Papua New Guinea,Papua New Guinean kina
PYF,French Polynesia,CFP franc
SLB,Solomon Islands,Solomon Islands dollar
WSM,Samoa,Samoan tala
TON,Tonga,Tongan paʻanga


> Note: Not all countries in Oceania were listed in the resulting inner join with currencies. We will use an anti-join to determine which countries were not included

In [77]:
%%sql
-- 3. Select fields
SELECT code, name
  -- 4. From Countries
  FROM countries
  -- 5. Where continent is Oceania
  WHERE continent='Oceania'
  	-- 1. And code not in
  	AND code NOT IN
  	-- 2. Subquery
  	(SELECT code
  	FROM currencies);

 * sqlite://
Done.


code,name
ASM,American Samoa
FJI,Fiji Islands
GUM,Guam
FSM,"Micronesia, Federated States of"
MNP,Northern Mariana Islands


In [78]:
%%sql
-- Select the city name
SELECT name
  -- Alias the table where city name resides
  FROM cities AS c1
  -- Choose only records matching the result of multiple set theory clauses
  WHERE country_code IN
(
    -- Select appropriate field from economies AS e
    SELECT e.code
    FROM economies AS e
    -- Get all additional (unique) values of the field from currencies AS c2  
    UNION
    SELECT c2.code
    FROM currencies AS c2
    -- Exclude those appearing in populations AS p
    EXCEPT
    SELECT p.country_code
    FROM populations AS p
);

 * sqlite://
Done.


name
Bucharest
Kaohsiung
New Taipei City
Taichung
Tainan
Taipei


# Subqueries

In this closing chapter, we'll explore how to use nested queries and we'll solve three challenge problems.

## Subqueries inside WHERE and SELECT clauses


### Average fert_rate

In [79]:
%%sql
SELECT AVG(fert_rate) 
    FROM states;

 * sqlite://
Done.


AVG(fert_rate)
2.2838461538461536


### Asian countries below average `fert_rate`

In [80]:
%%sql
SELECT name, fert_rate
    FROM states
    WHERE continent = 'Asia'


 * sqlite://
Done.


name,fert_rate
Brunei,1.96
India,2.43
Oman,2.75
Vietnam,1.7


In [81]:
%%sql
SELECT name, fert_rate
    FROM states
    WHERE continent = 'Asia' AND fert_rate < (
        SELECT AVG(fert_rate) FROM states);

 * sqlite://
Done.


name,fert_rate
Brunei,1.96
Vietnam,1.7


### Subqueries inside SELECT clauses - setup

In [82]:
%%sql
SELECT DISTINCT(continent) 
    FROM prime_ministers;

 * sqlite://
Done.


continent
Africa
Europe
Asia
North America
Oceania


### Subquery inside SELECT clause - complete

In [83]:
%%sql
SELECT DISTINCT(continent), (SELECT COUNT(*)
                             FROM states
                             WHERE prime_ministers.continent = states.continent) AS countries_num
FROM prime_ministers;

 * sqlite://
Done.


continent,countries_num
Africa,2
Europe,3
Asia,4
North America,1
Oceania,1


 which countries had high average life expectancies (at the country level) in 2015.

In [84]:
%%sql
-- Select average life_expectancy
SELECT AVG(life_expectancy)
  -- From populations
  FROM populations
-- Where year is 2015
WHERE year=2015;

 * sqlite://
Done.


AVG(life_expectancy)
71.67634158659767


In [85]:
%%sql
-- Select fields
SELECT *
  -- From populations
  FROM populations
-- Where life_expectancy is greater than
WHERE life_expectancy >
  -- 1.15 * subquery
  (1.15 * (SELECT AVG(life_expectancy)
  FROM populations
  WHERE year=2015))
   AND
   year=2015;

 * sqlite://
Done.


index,pop_id,country_code,year,fertility_rate,life_expectancy,size
21,21,AUS,2015,1.8330000000000002,82.4512195121951,23789752.0
67,376,CHE,2015,1.54,83.1975609756098,8281430.0
119,356,ESP,2015,1.32,83.38048780487809,46443994.0
129,134,FRA,2015,2.01,82.67073170731709,66538391.0
165,170,HKG,2015,1.195,84.2780487804878,7305700.0
187,174,ISL,2015,1.93,82.8609756097561,330815.0
191,190,ITA,2015,1.37,83.490243902439,60730582.0
197,194,JPN,2015,1.46,83.84365853658541,126958472.0
341,340,SGP,2015,1.24,82.5951219512195,5535002.0
365,374,SWE,2015,1.88,82.55121951219509,9799186.0


urban area population for only capital cities.

In [86]:
%%sql
-- 2. Select fields
SELECT name, country_code, urbanarea_pop
  -- 3. From cities
  FROM cities
-- 4. Where city name in the field of capital cities
WHERE name IN
  -- 1. Subquery
  (SELECT capital
   FROM countries)
ORDER BY urbanarea_pop DESC
LIMIT 10;

 * sqlite://
Done.


name,country_code,urbanarea_pop
Beijing,CHN,21516000
Dhaka,BGD,14543124
Tokyo,JPN,13513734
Moscow,RUS,12197596
Cairo,EGY,10230350
Kinshasa,COD,10130000
Jakarta,IDN,10075310
Seoul,KOR,9995784
Mexico City,MEX,8974724
Lima,PER,8852000


the top nine countries in terms of number of cities appearing in the `cities` table.

In [87]:
%%sql
SELECT countries.name AS country, COUNT(*) AS cities_num
  FROM cities
    INNER JOIN countries
    ON countries.code = cities.country_code
GROUP BY country
ORDER BY cities_num DESC, country
LIMIT 9;

 * sqlite://
Done.


country,cities_num
China,36
India,18
Japan,11
Brazil,10
Pakistan,9
United States,9
Indonesia,7
Russian Federation,7
South Korea,7


In [88]:
%%sql
SELECT name AS country,
  (SELECT COUNT(*)
   FROM cities
   WHERE countries.code = cities.country_code) AS cities_num
FROM countries
ORDER BY cities_num DESC, country
LIMIT 9;

 * sqlite://
Done.


country,cities_num
China,36
India,18
Japan,11
Brazil,10
Pakistan,9
United States,9
Indonesia,7
Russian Federation,7
South Korea,7


## Subquery inside FROM clause


### Build-up

In [91]:
%%sql
SELECT continent, MAX(women_parli_perc) AS max_perc
    FROM states
    GROUP BY continent
    ORDER BY continent;

 * sqlite://
Done.


continent,max_perc
Africa,14.9
Asia,24.0
Europe,39.6
North America,2.74
Oceania,32.74
South America,22.31


### Focusing on records in monarchs

In [93]:
%%sql
SELECT monarchs.continent
    FROM monarchs 
    INNER JOIN states   
    using(continent)
    ORDER BY continent;

 * sqlite://
Done.


continent
Asia
Asia
Asia
Asia
Asia
Asia
Asia
Asia
Europe
Europe


### Finishing off the subquery

In [95]:
%%sql
SELECT DISTINCT(monarchs.continent), MAX(women_parli_perc) max_perc
    FROM monarchs
    INNER JOIN states
    USING(continent)
    GROUP BY continent
    ORDER BY continent;

 * sqlite://
Done.


continent,max_perc
Asia,24.0
Europe,39.6


the number of languages spoken for each country, identified by the country's local name! (Note this may be different than the name field and is stored in the local_name field.)

In [97]:
%%sql
-- Select fields
SELECT countries.local_name, lang_num
  -- From countries
  FROM countries,
  	-- Subquery (alias as subquery)
  	(SELECT code, COUNT(*) lang_num
  	 FROM languages
  	 GROUP BY code) AS subquery
  -- Where codes match
  WHERE countries.code = subquery.code
-- Order by descending number of languages
ORDER BY lang_num DESC
LIMIT 10;

 * sqlite://
Done.


local_name,lang_num
Zambia,19
YeItyop´iya,16
Zimbabwe,16
Bharat/India,14
Nepal,14
South Africa,13
Mali,13
France,13
Angola,12
Malawi,12


### Advanced subquery

We can also nest multiple subqueries to answer even more specific questions.

for each of the six continents listed in 2015, We'll identify which country had the maximum inflation rate (and how high it was) using multiple subqueries. 

In [98]:
%%sql
-- Select fields
SELECT name, continent, inflation_rate
  -- From countries
  FROM countries
  	-- Join to economies
  	INNER JOIN economies
    -- Match on code
    USING(code)
-- Where year is 2015
WHERE year=2015
LIMIT 10;

 * sqlite://
Done.


name,continent,inflation_rate
Afghanistan,Asia,-1.5490000000000002
Angola,Africa,10.287
Albania,Europe,1.896
United Arab Emirates,Asia,4.07
Argentina,South America,None
Armenia,Asia,3.7310000000000003
Antigua and Barbuda,North America,0.9690000000000001
Australia,Oceania,1.4609999999999999
Austria,Europe,0.81
Azerbaijan,Asia,4.0489999999999995


In [100]:
%%sql
-- Select the maximum inflation rate as max_inf
SELECT continent, MAX(inflation_rate) AS max_inf
  -- Subquery using FROM (alias as subquery)
  FROM (
      SELECT name, continent, inflation_rate
      FROM countries
      INNER JOIN economies
      USING(code)
      WHERE year=2015) AS subquery
-- Group by continent
GROUP BY continent;

 * sqlite://
Done.


continent,max_inf
Africa,21.858
Asia,39.403
Europe,48.684
North America,7.524
Oceania,9.783999999999999
South America,121.738


In [101]:
%%sql
-- Select fields
SELECT name, continent, inflation_rate
  -- From countries
  FROM countries
	-- Join to economies
	INNER JOIN economies
	-- Match on code
	USING(code)
  -- Where year is 2015
  WHERE year = 2015
    -- And inflation rate in subquery (alias as subquery)
    AND inflation_rate IN (
        SELECT MAX(inflation_rate) AS max_inf
        FROM (
             SELECT name, continent, inflation_rate
             FROM countries
             INNER JOIN economies
             USING(code)
             WHERE year = 2015) AS subquery
      -- Group by continent
        GROUP BY continent);

 * sqlite://
Done.


name,continent,inflation_rate
Haiti,North America,7.524
Malawi,Africa,21.858
Nauru,Oceania,9.783999999999999
Ukraine,Europe,48.684
Venezuela,South America,121.738
Yemen,Asia,39.403


This code works since each of the six maximum inflation rate values occur only once in the 2015 data. 

2015 economic data for countries that do not have

- `gov_form` of `'Constitutional Monarchy'` or
- `'Republic'` in their `gov_form`.

Here, `gov_form` stands for the form of the government for each country.

In [103]:
%%sql
-- Select fields
SELECT code, inflation_rate, unemployment_rate
  -- From economies
  FROM economies
  -- Where year is 2015 and code is not in
  WHERE year = 2015 AND code NOT in
  	-- Subquery
  	(SELECT code
  	 FROM countries
  	 WHERE (gov_form = 'Constitutional Monarchy' OR gov_form LIKE '%Republic%'))
-- Order by inflation rate
ORDER BY inflation_rate
LIMIT 15;

 * sqlite://
Done.


code,inflation_rate,unemployment_rate
AFG,-1.5490000000000002,None
CHE,-1.14,3.178
PRI,-0.7509999999999999,12.0
ROU,-0.596,6.812
BRN,-0.423,6.9
TON,-0.28300000000000003,None
OMN,0.065,None
TLS,0.5529999999999999,None
BEL,0.62,8.492
CAN,1.1320000000000001,6.9


### Types of joins
- INNER JOIN
  -  Self-joins
- OUTER JOIN
  - LEFT JOIN
  - RIGHT JOIN
  - FULL JOIN
- CROSS JOIN
- Semi-join / Anti-join


### INNER JOIN vs LEFT JOIN

![image.png](datasets/images/inner-left-join.png "inner-left-join.png")

### RIGHT JOIN vs FULL JOIN


![image.png](datasets/images/right-full-join.png "right-full-join.png")

### Types of basic subqueries
- Subqueries inside WHERE clauses
- Subqueries inside SELECT clauses
- Subqueries inside FROM clauses

 the country names and other 2015 data in the `economies` table and the `countries` table for **Central American countries with an official language.**

In [105]:
%%sql
-- Select fields
SELECT DISTINCT c.name, e.total_investment, e.imports
  -- From table (with alias)
  FROM countries AS c
    -- Join with table (with alias)
    LEFT JOIN economies AS e
      -- Match on code
      ON c.code=e.code
      -- and code in Subquery
        AND c.code IN (
          SELECT l.code
          FROM  languages AS l
          WHERE official = 'true'
         )
  -- Where region and year are correct
  WHERE region = 'Central America' AND year = 2015
-- Order by field
ORDER BY c.name;

 * sqlite://
Done.


name,total_investment,imports


the average fertility rate for each region in 2015.

In [106]:
%%sql
-- Select fields
SELECT region, continent, AVG(fertility_rate) AS avg_fert_rate
  -- From left table
  FROM countries AS c1
    -- Join to right table
    INNER JOIN populations AS p
      -- Match on join condition
      ON c1.code = p.country_code
  -- Where specific records matching some condition
  WHERE year = 2015
-- Group appropriately
GROUP BY region, continent
-- Order appropriately
ORDER BY avg_fert_rate;

 * sqlite://
Done.


region,continent,avg_fert_rate
Southern Europe,Europe,1.4261000000000001
Eastern Europe,Europe,1.4908888888888887
Baltic Countries,Europe,1.6033333333333333
Eastern Asia,Asia,1.6207142857142856
Western Europe,Europe,1.6324999999999998
North America,North America,1.7657500000000002
British Islands,Europe,1.875
Nordic Countries,Europe,1.8933333333333333
Australia and New Zealand,Oceania,1.9115000000000002
Caribbean,North America,1.9505714285714277


It seems that the average fertility rate is lowest in Southern Europe and highest in Central Africa.

the top 10 capital cities in Europe and the Americas in terms of a calculated percentage using `city_proper_pop` and `metroarea_pop` in cities.

In [107]:
%%sql
-- Select fields
SELECT name, country_code, city_proper_pop, metroarea_pop,  
      -- Calculate city_perc
     city_proper_pop  / metroarea_pop * 100 AS city_perc
  -- From appropriate table
  FROM cities
  -- Where 
  WHERE name IN
    -- Subquery
    (SELECT capital
     FROM countries
     WHERE (continent = 'Europe'
        OR continent LIKE '%America'))
       AND metroarea_pop IS NOT NULL
-- Order appropriately
ORDER BY city_perc DESC
-- Limit amount
LIMIT 10;

 * sqlite://
Done.


name,country_code,city_proper_pop,metroarea_pop,city_perc
Lima,PER,8852000,10750000.0,82.34418604651162
Bogota,COL,7878783,9800000.0,80.39574489795919
Moscow,RUS,12197596,16170000.0,75.43349412492269
Vienna,AUT,1863881,2600000.0,71.68773076923077
Montevideo,URY,1305082,1947604.0,67.00961797162051
Caracas,VEN,1943901,2923959.0,66.48181455348724
Rome,ITA,2877215,4353775.0,66.08552348249508
Brasilia,BRA,2556149,3919864.0,65.21014504584853
London,GBR,8673713,13879757.0,62.491821722815466
Budapest,HUN,1759407,2927944.0,60.09018615110126
